### Goal: calculate the avilability for all properties

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
#Importing all the datasets needed
#comb_reviews = pd.read_csv('q6_comb_reviews.csv')
pdx_cal = pd.read_csv('PDX_calendar.csv')
la_cal = pd.read_csv('la_calendar.csv')
sd_cal = pd.read_csv('sd_calendar.csv')
salem_cal = pd.read_csv('salem_calendar.csv')

In [195]:
#Combining all the calendars
df_list = [pdx_cal, la_cal, sd_cal, salem_cal]
comb_cal = pd.concat(df_list, ignore_index = True)

#Testing concat
indiv_lengths = len(pdx_cal) + len(la_cal) + len(sd_cal) + len(salem_cal)
if len(comb_cal) == indiv_lengths:
    print("Concat worked correctly. New len: "+str(len(comb_cal)))
else:
    print("Concat did not merge right")
    
print(comb_cal.head())

Concat worked correctly. New len: 21966856
   listing_id        date available   price adjusted_price  minimum_nights  \
0       12899  2023-03-23         f  $80.00         $80.00             3.0   
1       12899  2023-03-24         f  $80.00         $80.00             3.0   
2       12899  2023-03-25         f  $80.00         $80.00             3.0   
3       12899  2023-03-26         f  $80.00         $80.00             3.0   
4       12899  2023-03-27         f  $80.00         $80.00             3.0   

   maximum_nights  
0          1125.0  
1          1125.0  
2          1125.0  
3          1125.0  
4          1125.0  


In [176]:
'''#Making a toy dataset
toy_df = comb_cal[comb_cal['listing_id']==29967] #originally used 29967
print(len(toy_df))
print(toy_df.head())'''

365
          listing_id        date available    price adjusted_price  \
17164916       29967  2023-03-25         f  $225.00        $225.00   
17164917       29967  2023-03-26         f  $225.00        $225.00   
17164918       29967  2023-03-27         f  $225.00        $225.00   
17164919       29967  2023-03-28         f  $225.00        $225.00   
17164920       29967  2023-03-29         f  $225.00        $225.00   

          minimum_nights  maximum_nights  
17164916             4.0           365.0  
17164917             4.0           365.0  
17164918             4.0           365.0  
17164919             4.0           365.0  
17164920             4.0           365.0  


In [ ]:
'''Steps:
    1. import the calendar information for all the locations
    2. Create the row indices based on the gaps and islands methods
    3. Save the table, and remove all gaps
    4. Create a calculation with the length of each island. Append it to the islands table
    4. for each listing, for each date:
        4.a check if the date is in the islands table...if so:
        4.b check the length of the island via the length column - if it's greater than the min days of the current day proceed
        4.c assuming we can proceed, check the distance of the current date from the ends of the island
        4.d if the distance from the ends is greater than the min stay then set the availability to true
        
NEED TO MAKE THIS WORK WITH MULTIPLE LISTINGS - CURRENT CODE ASSUMES THERE"S ONLY 1

NEED TO SORT DATE DESCENDING, DO SAME LOGIC AGAIN TO CREATE A DESCENDING ROW ID
POS ROWID - REAL ROW ID = LENGTH SINCE START OF ISLAND
NEG ROW ID - REAL ROW ID = LENGTH SINCE END OF ISLAND'''

In [230]:
#SETTING THE DF WE WILL BE WORKING WITH
df = comb_cal #toy_df.copy()
len(df)

21966856

In [231]:
#Creating the FORWARD gaps and islands method - need to add mask, mask cumsum
df = df.sort_values(by = ['listing_id','date'], ascending=True)
df['inverse_mask'] = np.where(df['available'] == 't', False, True)
df['ASC_cumsum'] = df['inverse_mask'].cumsum()
df['ASC_row_number'] = df.reset_index().index

#Checking range with changes over t/f (index based on toy_df)
df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,inverse_mask,ASC_cumsum,ASC_row_number
17164551,6,2023-03-25,f,$349.00,$349.00,10.0,30.0,True,1,0
17164552,6,2023-03-26,f,$349.00,$349.00,10.0,30.0,True,2,1
17164553,6,2023-03-27,f,$349.00,$349.00,10.0,30.0,True,3,2
17164554,6,2023-03-28,f,$349.00,$349.00,10.0,30.0,True,4,3
17164555,6,2023-03-29,f,$349.00,$349.00,10.0,30.0,True,5,4


In [232]:
#SORT THE DATA IN DATE DESCENDING & CALC DESC_CUMSUM
df = df.sort_values(by = ['listing_id','date'], ascending= False)
df['DESC_cumsum'] = df['inverse_mask'].cumsum()
df['DESC_row_number'] = df.reset_index().index

#Flipping the dataframe back around
df = df.sort_values(by = ['listing_id','date'], ascending= True)

In [233]:
#Need a table with asc_cumsum, the length, the cume length
df_t = df[df['available']=='t']

#Creating information on the islands' length, and cumulative length
isl_info = df_t.groupby('ASC_cumsum').size().reset_index()
isl_info = isl_info.rename({isl_info.columns[1]:'Isl_len'}, axis = 1)
isl_info['ASC_cum_len'] = isl_info['Isl_len'].cumsum().shift().fillna(0)
isl_info['DESC_cum_len'] = isl_info.loc[::-1,'Isl_len'].cumsum()[::-1].shift(-1).fillna(0)

isl_info

,ASC_cumsum,Isl_len,ASC_cum_len,DESC_cum_len
0,412,318,0.0,11787073.0
1,452,14,318.0,11787059.0
2,512,251,332.0,11786808.0
3,515,362,583.0,11786446.0
4,880,365,945.0,11786081.0
...,...,...,...,...
204338,10179212,276,11786555.0,560.0
204339,10179296,4,11786831.0,556.0
204340,10179297,276,11786835.0,280.0
204341,10179381,4,11787111.0,276.0


In [234]:
#Adding merging the island info table and the big df for calculations
merge_df = pd.merge(df, isl_info, how = 'left', on = 'ASC_cumsum')
merge_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,inverse_mask,ASC_cumsum,ASC_row_number,DESC_cumsum,DESC_row_number,Isl_len,ASC_cum_len,DESC_cum_len
0,6,2023-03-25,f,$349.00,$349.00,10.0,30.0,True,1,0,10179465,21966855,NaN,NaN,NaN
1,6,2023-03-26,f,$349.00,$349.00,10.0,30.0,True,2,1,10179464,21966854,NaN,NaN,NaN
2,6,2023-03-27,f,$349.00,$349.00,10.0,30.0,True,3,2,10179463,21966853,NaN,NaN,NaN
3,6,2023-03-28,f,$349.00,$349.00,10.0,30.0,True,4,3,10179462,21966852,NaN,NaN,NaN
4,6,2023-03-29,f,$349.00,$349.00,10.0,30.0,True,5,4,10179461,21966851,NaN,NaN,NaN


In [239]:
#Adding distance from beginning of island col
merge_df['days_to_beg_isl'] = np.where(merge_df['available']=='t', 
                                      merge_df['ASC_row_number'] - merge_df['ASC_cumsum'] - merge_df['ASC_cum_len'],
                                      0)
merge_df.loc[:,['date', 'available','ASC_cumsum', 'ASC_row_number', 'ASC_cum_len','days_to_beg_isl']]

,date,available,ASC_cumsum,ASC_row_number,ASC_cum_len,days_to_beg_isl
0,2023-03-25,f,1,0,NaN,0.0
1,2023-03-26,f,2,1,NaN,0.0
2,2023-03-27,f,3,2,NaN,0.0
3,2023-03-28,f,4,3,NaN,0.0
4,2023-03-29,f,5,4,NaN,0.0
...,...,...,...,...,...,...
21966851,2024-03-19,f,10179461,21966851,NaN,0.0
21966852,2024-03-20,f,10179462,21966852,NaN,0.0
21966853,2024-03-21,f,10179463,21966853,NaN,0.0
21966854,2024-03-22,f,10179464,21966854,NaN,0.0


In [240]:
#Adding distance from end of island col
merge_df['days_to_end_isl'] = np.where(merge_df['available']=='t', 
                                      merge_df['DESC_row_number'] - merge_df['DESC_cumsum'] - merge_df['DESC_cum_len'],
                                      0)
merge_df.loc[:,['date', 'available','days_to_beg_isl','days_to_end_isl']]

,date,available,days_to_beg_isl,days_to_end_isl
0,2023-03-25,f,0.0,0.0
1,2023-03-26,f,0.0,0.0
2,2023-03-27,f,0.0,0.0
3,2023-03-28,f,0.0,0.0
4,2023-03-29,f,0.0,0.0
...,...,...,...,...
21966851,2024-03-19,f,0.0,0.0
21966852,2024-03-20,f,0.0,0.0
21966853,2024-03-21,f,0.0,0.0
21966854,2024-03-22,f,0.0,0.0


In [242]:
#Adding the true availability
merge_df['true_availability'] = np.where((merge_df['days_to_beg_isl']>= merge_df['minimum_nights']) 
                                         | (merge_df['days_to_end_isl']>= merge_df['minimum_nights']),
                                        True, False)
merge_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,inverse_mask,ASC_cumsum,ASC_row_number,DESC_cumsum,DESC_row_number,Isl_len,ASC_cum_len,DESC_cum_len,days_to_beg_isl,days_to_end_isl,true_availability
0,6,2023-03-25,f,$349.00,$349.00,10.0,30.0,True,1,0,10179465,21966855,NaN,NaN,NaN,0.0,0.0,False
1,6,2023-03-26,f,$349.00,$349.00,10.0,30.0,True,2,1,10179464,21966854,NaN,NaN,NaN,0.0,0.0,False
2,6,2023-03-27,f,$349.00,$349.00,10.0,30.0,True,3,2,10179463,21966853,NaN,NaN,NaN,0.0,0.0,False
3,6,2023-03-28,f,$349.00,$349.00,10.0,30.0,True,4,3,10179462,21966852,NaN,NaN,NaN,0.0,0.0,False
4,6,2023-03-29,f,$349.00,$349.00,10.0,30.0,True,5,4,10179461,21966851,NaN,NaN,NaN,0.0,0.0,False


In [243]:
#Checking output
merge_df.loc[((merge_df['listing_id'] == 29967) & (merge_df['true_availability'] == True))]

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,inverse_mask,ASC_cumsum,ASC_row_number,DESC_cumsum,DESC_row_number,Isl_len,ASC_cum_len,DESC_cum_len,days_to_beg_isl,days_to_end_isl,true_availability
13942,29967,2023-06-05,t,$275.00,$275.00,4.0,365.0,False,7115,13942,10172350,21952913,11.0,6827.0,11780553.0,0.0,10.0,True
13943,29967,2023-06-06,t,$275.00,$275.00,4.0,365.0,False,7115,13943,10172350,21952912,11.0,6827.0,11780553.0,1.0,9.0,True
13944,29967,2023-06-07,t,$275.00,$275.00,4.0,365.0,False,7115,13944,10172350,21952911,11.0,6827.0,11780553.0,2.0,8.0,True
13945,29967,2023-06-08,t,$275.00,$275.00,4.0,365.0,False,7115,13945,10172350,21952910,11.0,6827.0,11780553.0,3.0,7.0,True
13946,29967,2023-06-09,t,$275.00,$275.00,4.0,365.0,False,7115,13946,10172350,21952909,11.0,6827.0,11780553.0,4.0,6.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14230,29967,2024-03-19,t,$225.00,$225.00,4.0,365.0,False,7285,14230,10172180,21952625,83.0,6867.0,11780441.0,78.0,4.0,True
14231,29967,2024-03-20,t,$225.00,$225.00,4.0,365.0,False,7285,14231,10172180,21952624,83.0,6867.0,11780441.0,79.0,3.0,True
14232,29967,2024-03-21,t,$225.00,$225.00,4.0,365.0,False,7285,14232,10172180,21952623,83.0,6867.0,11780441.0,80.0,2.0,True
14233,29967,2024-03-22,t,$225.00,$225.00,4.0,365.0,False,7285,14233,10172180,21952622,83.0,6867.0,11780441.0,81.0,1.0,True


In [244]:
#Keep only the relevent columns
final_avail_df = merge_df.loc[:,['listing_id','date', 'available', 'minimum_nights','days_to_beg_isl','days_to_end_isl','true_availability']]
final_avail_df.head()

,listing_id,date,available,minimum_nights,days_to_beg_isl,days_to_end_isl,true_availability
0,6,2023-03-25,f,10.0,0.0,0.0,False
1,6,2023-03-26,f,10.0,0.0,0.0,False
2,6,2023-03-27,f,10.0,0.0,0.0,False
3,6,2023-03-28,f,10.0,0.0,0.0,False
4,6,2023-03-29,f,10.0,0.0,0.0,False


In [245]:
#Saving output
final_avail_df.to_csv('final_avail_data.csv')